In [ ]:
#write a simple python program to Build an ensemble model to classify images
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the image data
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple CNN model
def build_cnn_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Define a model using a pretrained base (VGG16)
def build_pretrained_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Build base models
cnn_model = build_cnn_model()
pretrained_model = build_pretrained_model()

# Compile the base models
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train base models
cnn_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)
pretrained_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

# Ensemble model: average predictions from both models
class EnsembleModel(tf.keras.Model):
    def __init__(self, models):
        super(EnsembleModel, self).__init__()
        self.models = models

    def call(self, inputs):
        predictions = [model(inputs) for model in self.models]
        return tf.reduce_mean(predictions, axis=0)

# Create an ensemble model
ensemble_model = EnsembleModel([cnn_model, pretrained_model])

# Evaluate the ensemble model on test data
ensemble_predictions = ensemble_model(x_test)
ensemble_accuracy = tf.keras.metrics.categorical_accuracy(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {tf.reduce_mean(ensemble_accuracy).numpy():.2f}")


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 60s 82ms/step - accuracy: 0.3414 - loss: 1.8036 - val_accuracy: 0.5372 - val_loss: 1.2972
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 84s 85ms/step - accuracy: 0.5616 - loss: 1.2451 - val_accuracy: 0.5970 - val_loss: 1.1371
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 79s 81ms/step - accuracy: 0.6195 - loss: 1.0895 - val_accuracy: 0.6376 - val_loss: 1.0515
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 80ms/step - accuracy: 0.6547 - loss: 0.9911 - val_accuracy: 0.6176 - val_loss: 1.1024
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - accuracy: 0.6710 - loss: 0.9402 - val_accuracy: 0.6750 - val_loss: 0.9468
Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 613s 869ms/step - accuracy: 0.4342 - loss: 1.6289 - val_accuracy: 0.5650 - val_loss: 1.2383
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 619s 865ms/step - accuracy: 0.5694 - loss: 1.2388 - val_accuracy: 0.5826 - val_loss: 1.1926
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 621